In [1]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
from matplotlib import pyplot as plt

from lib.generate_data import Sampler, DummyData, MultitaskSparseParity

## Set up data

In [4]:
n_data_bits = 100
# n_control_bits = 500
n_control_bits = 5
k = 3
alpha = 0.4

# sampler: Sampler = MultitaskSparseParity(n_control_bits, n_data_bits, alpha=alpha)
sampler: Sampler = DummyData(n_control_bits + n_data_bits)

In [14]:
sampler.generate_data(2)

/home/rmkj/Documents/Research/quantization-neural-scaling/lib/generate_data.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.cat((torch.tensor(x_control), torch.tensor(x_data)), axis=1)


(tensor([[0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0.,
          1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0.,
          0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0.,
          0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 1.,
          1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
          0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0.],
         [0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
          0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
          0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
          1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0.,
          1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0.,
          0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1.]],
        dtype=torch.float64),
 tensor

In [ ]:

%timeit sampler.generate_data(20000)

## Train Network

In [ ]:
batch_size = 20000
training_size = 1e5

n_hidden = 200
lr = 1e-3
n_epochs = 1000
optimizer_func = lambda model: torch.optim.Adam(model.parameters(), lr=lr)
loss_func = torch.nn.BCELoss()

In [ ]:
class TinyModel(torch.nn.Module):

    def __init__(self, n_hidden: int):
        super(TinyModel, self).__init__()

        self.linear1 = torch.nn.Linear(n_control_bits + n_data_bits, n_hidden)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(n_hidden, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.sigmoid(x)
        return x

model = TinyModel(n_hidden)
optimizer = optimizer_func(model)

In [ ]:
for epoch in range(n_epochs):
    for i in range(int(training_size // batch_size)):
        X_batch, y_batch = sampler.generate_data(batch_size)


        y_pred = model(X_batch.float())
        loss = loss_func(y_pred, y_batch[:, None].float())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
    print(f"Epoch: {epoch} loss: {loss.item()}")